# ADS 509 Module 1: APIs and Web Scraping

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Twitter API Pull

In [23]:
# pip install tweepy

In [24]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter


In [59]:
import seaborn as sns
import pandas as pd
import numpy as np
import os
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import requests
from bs4 import BeautifulSoup
import shutil

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [26]:
from api_keys import api_key, api_key_secret, access_token, access_token_secret 

In [27]:
auth = tweepy.AppAuthHandler(api_key, api_key_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

print ('API Host', api.host)
#print ('API Version', api.api_root)

API Host api.twitter.com


In [28]:
# dir(api)

## Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull all the follower IDs for @katymck.
1. Explore the user object, which gives us information about Twitter users. 
1. Pull some user objects for the followers. 
1. Pull the last few tweets by @katymck.


In [29]:
# I had to change api.followers_ids to api.get_followers_ids since it returned the error that it had no followers_ids??

handle = "37chandler"

followers = []

for page in tweepy.Cursor(api.get_follower_ids,
                          # This is how we will get around the issue of not being able to grab all ids at once
                          # Once the rate limit is hit, we will be  that we must wait 15 mins (900 secs)
#                           wait_on_rate_limit=True, 
#                           wait_on_rate_limit_notify=True, 
#                           compression=True,
                          screen_name=handle).pages():

    # The page variable comes back as a list, so we have to use .extend rather than .append
    followers.extend(page)
        
        
print(f"Here are the first five follower ids for {handle} out of the {len(followers)} total.")
followers[:5]

# Section Complete

Here are the first five follower ids for 37chandler out of the 189 total.


[257285645, 1469785454576820225, 1181131341687066624, 257686741, 2306579816]

We have the follower IDs, which are unique numbers identifying the user, but we'd like to get some more information on these users. Twitter allows us to pull "fully hydrated user objects", which is a fancy way of saying "all the information about the user". Let's look at user object for our starting handle.

In [30]:
user = api.get_user(user_id=8813602) 
print(user._json)

# Section Complete
# I needed to do user_id = because it no longer displays the users, it returns an error that it expects 1 but 2 were provided.

{'id': 8813602, 'id_str': '8813602', 'name': 'Stacey Burns', 'screen_name': 'WentRogue', 'location': 'Minneapolis Witch District', 'profile_location': None, 'description': 'Once inspired the NYT Science Times to "try harder." And now I tweet about my cats. Cis; she/her. I used to be someone’s big sister. I am still not OK.', 'url': 'https://t.co/CgQ2gGvTks', 'entities': {'url': {'urls': [{'url': 'https://t.co/CgQ2gGvTks', 'expanded_url': 'https://unrestrictmn.org/petition-reproductive-freedom/?ms=SBtwitter', 'display_url': 'unrestrictmn.org/petition-repro…', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 10554, 'friends_count': 4453, 'listed_count': 278, 'created_at': 'Tue Sep 11 16:25:31 +0000 2007', 'favourites_count': 141964, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': True, 'statuses_count': 184673, 'lang': None, 'status': {'created_at': 'Mon May 16 21:42:28 +0000 2022', 'id': 1526317181767667717, 'id_str': '

In [31]:
print(f"{len(user._json.keys())} individual fields are being returned in the _json portion of the user object")

45 individual fields are being returned in the _json portion of the user object


Now a few questions for you about the user object.

Q: How many fields are being returned in the \_json portion of the user object? 

A: There are 45 fields being returned, they can be seen above for the user I chose from the list of followers. 

---

Q: Are any of the fields within the user object non-scaler? TK correct term

A: I am not sure what TK correct term means, but there appears to be non-scaler fields that are not integers or other scaler fields. This can be seen in the above listing. 

---

Q: How many friends, followers, favorites, and statuses does this user have? 

A: Friends: 4434, Followers: 10523, Favorites: 141585, Statuses: 184339 **This section seems to keep changing when I run each day. 


We can map the follower IDs onto screen names by accessing the screen_name key within the user object. Modify the code below to also print out how many people the follower is following and how many followers they have. 

In [32]:
# I had to add in user_id= to be able to run properly, update from tweepy

ids_to_lookup = followers[:10]

for user_obj in api.lookup_users(user_id=ids_to_lookup) :
    print(f"{handle} is followed by {user_obj.screen_name}")
    
    # Add code here to print out friends and followers of `handle`
    print(f"{user_obj.screen_name} is following {user_obj._json['friends_count']} people")
    print(f"{user_obj.screen_name} has {user_obj._json['followers_count']} followers\n")
    
# Section Complete

37chandler is followed by HicSvntDraconez
HicSvntDraconez is following 1546 people
HicSvntDraconez has 33 followers

37chandler is followed by JohnOCo70713197
JohnOCo70713197 is following 8 people
JohnOCo70713197 has 1 followers

37chandler is followed by CodeGradeCom
CodeGradeCom is following 2711 people
CodeGradeCom has 388 followers

37chandler is followed by cleverhoods
cleverhoods is following 2774 people
cleverhoods has 3371 followers

37chandler is followed by PaulNaish78
PaulNaish78 is following 19266 people
PaulNaish78 has 19524 followers

37chandler is followed by mplsFietser
mplsFietser is following 2651 people
mplsFietser has 2783 followers

37chandler is followed by echallstrom
echallstrom is following 457 people
echallstrom has 304 followers

37chandler is followed by byler_t117
byler_t117 is following 440 people
byler_t117 has 48 followers

37chandler is followed by Community_Owner
Community_Owner is following 47 people
Community_Owner has 31 followers

37chandler is fol

Although you won't need it for this assignment, individual tweets (called "statuses" in the API) can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the `status` object and marvel in the richness of the data that is available. 


In [33]:
tweet_count = 0

for status in tweepy.Cursor(api.user_timeline, screen_name=handle).items():
    tweet_count += 1
    
    print(f"The tweet was tweeted at {status.created_at}.")
    print(f"The original tweet has been retweeted {status.retweet_count} times.")
    
    clean_status = status.text
    clean_status = clean_status.replace("\n"," ")
    
    print(f"{clean_status}")
    print("\n"*2)
        
    if tweet_count > 10 :
        break

#Section Complete

The tweet was tweeted at 2022-05-13 23:46:46+00:00.
The original tweet has been retweeted 5 times.
RT @johnhollinger: @NateSilver538 Atlanta still leads the nation in "further West than you think"



The tweet was tweeted at 2022-05-13 12:03:18+00:00.
The original tweet has been retweeted 1478 times.
RT @tomscocca: It was helpful to talk with @pareene about the experience of turning away from one of the most comfortable default beliefs o…



The tweet was tweeted at 2022-05-12 14:41:18+00:00.
The original tweet has been retweeted 187 times.
RT @JamesTateHill: I Actually Thought You Had Dropped the Class: A Memoir of Your Final Grade



The tweet was tweeted at 2022-05-12 12:19:43+00:00.
The original tweet has been retweeted 0 times.
@BluehairCoffee @WedgeLIVE I try to always take a pic of it. https://t.co/7i4nIgBKFM



The tweet was tweeted at 2022-05-12 03:18:56+00:00.
The original tweet has been retweeted 0 times.
@LiberalwKnives @WedgeLIVE @BluehairCoffee It seemed deep enough to to

## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We must first get the follower IDs, then we will be able to "hydrate" the IDs, pulling the user objects for them. Once we have those user objects we will extract some fields that we can use in future analyses. 


The Twitter API only allows users to make 15 requests per 15 minutes when pulling followers. Each request allows you to gather 5000 follower ids. Tweepy will grab the 15 requests quickly then wait 15 minutes, rather than slowly pull the requests over the time period. Before we start grabbing follower IDs, let's first just check how long it would take to pull all of the followers. To do this we use the `followers_count` item from the user object. 

In [34]:
# I needed to update the screen_name= in order to run, update to tweepy
# I'm also putting the handles in a list to iterate through below

handles = ['rushtheband','thekillers']

# This will iterate through each Twitter handle that we're collecting from
for screen_name in handles:
    
    # Tells Tweepy we want information on the handle we're collecting from
    # The next line specifies which information we want, which in this case is the number of followers 
    user = api.get_user(screen_name=screen_name) 
    followers_count = user.followers_count

    # Let's see roughly how long it will take to grab all the follower IDs. 
    print(f'''
    @{screen_name} has {followers_count} followers. 
    That will take roughly {followers_count/(5000*15*4):.2f} hours to pull the followers.
    ''')

# Section Complete


    @rushtheband has 302858 followers. 
    That will take roughly 1.01 hours to pull the followers.
    

    @thekillers has 4781598 followers. 
    That will take roughly 15.94 hours to pull the followers.
    


As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [35]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")
    
# Section Complete

In this following cells, use the `api.followers_ids` (and the `tweepy.Cursor` functionality) to pull some of the followers for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

In [36]:
num_followers_to_pull = 60*1000 # feel free to use this to limit the number of followers you pull.

In [37]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for idx, handle in enumerate(handles):

    output_file = "./twitter/"+handle + "_followers.txt"
    user = api.get_user(screen_name=handle) 
    followers_count = user.followers_count
    # Pull and store the follower IDs
    followers = []
    
    for page in tweepy.Cursor(api.get_follower_ids,
    #                           wait_on_rate_limit=True, 
    #                           wait_on_rate_limit_notify=True, 
    #                           compression=True,
                              count = int(num_followers_to_pull/15),
                              screen_name=handle).pages():
        followers.extend(page)
        # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response
        if len(followers) == num_followers_to_pull:
            break
    print(f"Pulled {len(followers)} followers of {handle}")
    
    # Write the IDs to the output file in the `twitter` folder.
    with open(output_file,'w') as f:
        for follower in followers:
            f.write(str(follower) + "\n")
    print(f"Wrote {len(followers)} followers of {handle} to file: {output_file}")        
    
    # Sleep before the next pull
    if idx != len(handles)-1:
        time.sleep(60*15)  
        
    
         
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)

# Section Complete

Rate limit reached. Sleeping for: 546


Pulled 60000 followers of rushtheband
Wrote 60000 followers of rushtheband to file: ./twitter/rushtheband_followers.txt
Pulled 60000 followers of thekillers
Wrote 60000 followers of thekillers to file: ./twitter/thekillers_followers.txt
0:24:16.508331


Now that you have your follower ids, gather some information that we can use in future assignments on them. Using the `lookup_users` function, pull the user objects for your followers. These requests are limited to 900 per 15 minutes, but you can request 100 users at a time. At 90,000 users per 15 minutes, the rate limiter on pulls might be bandwidth rather than API limits. 

Extract the following fields from the user object: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

These can all be accessed via these names in the object. Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 


In [38]:
handles = ['rushtheband','thekillers']

In [42]:
# in this cell, do the following
# 1. Set up a data frame or dictionary to hold the user information
# 2. Use the `lookup_users` api function to pull sets of 100 users at a time
# 3. Store the listed fields in your data frame or dictionary.
# 4. Write the user information in tab-delimited form to the follower data text file. 

import json

for handle in handles:
    input_file = "./twitter/"+handle + "_followers.txt"
    followers_id = [int(x) for x in open(input_file,'r').read().split('\n') if x]
    users_count = len(followers_id)
    #1 Set up Dataframe:
    my_list_of_dicts = []
    try:
        print(f"Getting user information for followers of {handle}")
        # 2. Use the `lookup_users` api function to pull sets of 100 users at a time
        for i in range(int(users_count / 100) + 1):
            each_json_tweet = api.lookup_users(user_id=followers_id[i*100:min((i+1)*100, users_count)])
            # 3. Store the listed fields in your data frame or dictionary.
            fields = ['screen_name','name','id','location','followers_count','friends_count','description']
            user_data = [{key: x._json[key] for key in fields} for x in each_json_tweet]
#             print(user_data)
            my_list_of_dicts.extend(user_data)
#             print('getting users batch:', i)
    except tweepy.errors.TweepyException as e:
        print('Something went wrong, quitting...', e)
        time.sleep(60*15)
    output_file = "./twitter/"+handle +"_follower_data.txt"
    # 4. Write the user information in tab-delimited form to the follower data text file. 
    with open(output_file, 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
        
# Section Complete

Getting user information for followers of rushtheband
Something went wrong, quitting... 404 Not Found
17 - No user matches for specified terms.
Getting user information for followers of thekillers
Something went wrong, quitting... 404 Not Found
17 - No user matches for specified terms.


One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. Here's an example of how you might do this. 

In [43]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description)
clean_description

' Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well. '

---

# Lyrics Scrape

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [52]:
artists = {'rush':"https://www.azlyrics.com/r/rush.html",
           'killers':"https://www.azlyrics.com/k/killers.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

# Section Complete

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: yes it appears that the scraping we are doing is allowed since I have the outputs of the song title and lyrics.  


In [53]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    soup = BeautifulSoup(r.text, 'html.parser')
    urls = []
    for link in soup.find_all('a'):
        url = link.get('href')
        if url and (artist in url) and ('/lyrics/' in url):
            full_url = 'https://www.azlyrics.com'+link.get('href')
            lyrics_pages[artist].append(full_url)
            
# Section Complete

In [54]:
lyrics_pages

defaultdict(list,
            {'rush': ['https://www.azlyrics.com/lyrics/rush/findingmyway.html',
              'https://www.azlyrics.com/lyrics/rush/needsomelove.html',
              'https://www.azlyrics.com/lyrics/rush/takeafriend.html',
              'https://www.azlyrics.com/lyrics/rush/hereagain.html',
              'https://www.azlyrics.com/lyrics/rush/whatyouredoing.html',
              'https://www.azlyrics.com/lyrics/rush/inthemood.html',
              'https://www.azlyrics.com/lyrics/rush/beforeandafter.html',
              'https://www.azlyrics.com/lyrics/rush/workingman.html',
              'https://www.azlyrics.com/lyrics/rush/anthem.html',
              'https://www.azlyrics.com/lyrics/rush/bestican.html',
              'https://www.azlyrics.com/lyrics/rush/beneathbetweenandbehind.html',
              'https://www.azlyrics.com/lyrics/rush/bytorandthesnowdog.html',
              'https://www.azlyrics.com/lyrics/rush/flybynight.html',
              'https://www.azlyrics.co

Let's make sure we have enough lyrics pages to scrape. 

In [55]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [56]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For rush we have 171.
The full pull will take for this artist will take 0.47 hours.
For killers we have 137.
The full pull will take for this artist will take 0.38 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [57]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [60]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [61]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :
    
    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    if not os.path.isdir(f"lyrics/{artist}/"): 
        os.mkdir(f'lyrics/{artist}/') 
    # 2. Iterate over the lyrics pages
    for artist_lyric_page in lyrics_pages[artist][:20]:
        print(artist_lyric_page)
        title = artist_lyric_page.split()
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        r = requests.get(artist_lyric_page)
        time.sleep(5 + 10*random.random())
    # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(r.text, 'html.parser')
        titles = []
        for text in soup.find_all('b'):
            if text and "".join([ch for ch in text.string.lower() if ch.isalpha()]) in artist_lyric_page:
                title = text.string.strip("\"")
                print(f"Extract title: {title}")
                titles.append(title)
        lyrics = []
        for divTag in soup.find_all("div", {"class": "col-xs-12 col-lg-8 text-center"}):
            for tag in divTag.find_all("div", {"class": ""}):
                lyric = tag.text.strip("\n")
                if lyric:
                    print(f"Extract lyric.")
                    lyrics.append(lyric)
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
        for title, lyric in zip(set(titles),lyrics):
            content = title+"\n"+"\n"+lyric
            output_file = generate_filename_from_link(artist_lyric_page)
            with open(f'lyrics/{artist}/'+output_file,'w') as f:
                f.write(content)
            print(f"Finished writing the lyric of {title} by {artist}")
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist


https://www.azlyrics.com/lyrics/rush/findingmyway.html
Extract title: Finding My Way
Extract title: Rush
Extract lyric.
Finished writing the lyric of Finding My Way by rush
https://www.azlyrics.com/lyrics/rush/needsomelove.html
Extract title: Need Some Love
Extract title: Rush
Extract lyric.
Finished writing the lyric of Need Some Love by rush
https://www.azlyrics.com/lyrics/rush/takeafriend.html
Extract title: Take A Friend
Extract title: Rush
Extract lyric.
Finished writing the lyric of Take A Friend by rush
https://www.azlyrics.com/lyrics/rush/hereagain.html
Extract title: Here Again
Extract title: Rush
Extract lyric.
Finished writing the lyric of Here Again by rush
https://www.azlyrics.com/lyrics/rush/whatyouredoing.html
Extract title: What You're Doing
Extract title: Rush
Extract lyric.
Finished writing the lyric of What You're Doing by rush
https://www.azlyrics.com/lyrics/rush/inthemood.html
Extract title: In The Mood
Extract title: Rush
Extract lyric.
Finished writing the lyric 

In [62]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.14 hours.


---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [63]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [64]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: thekillers and rushtheband.


In [67]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_follower_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_data_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")

        
# Section Complete, however, it doesnt return me any common words. I am unsure how to fix this as the moment because
# everything appears to be working correctly.

We see 59999 in your follower file for thekillers, assuming a header row.
In the follower data file (thekillers_follower_data.txt) for thekillers, we have these columns:
[

We have 539965 data rows for thekillers in the follower data file.
For thekillers we have 0 unique locations.
For thekillers we have 0 words in the descriptions.
Here are the five most common words:
[]

----------------------------------------

We see 59999 in your follower file for rushtheband, assuming a header row.
In the follower data file (rushtheband_follower_data.txt) for rushtheband, we have these columns:
[

We have 539938 data rows for rushtheband in the follower data file.
For rushtheband we have 0 unique locations.
For rushtheband we have 0 words in the descriptions.
Here are the five most common words:
[]

----------------------------------------



## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [68]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For killers we have 20 files.
For killers we have roughly 5247 words, 771 are unique.
For rush we have 20 files.
For rush we have roughly 3562 words, 878 are unique.


In [69]:
# End of Homework.